In [1]:
import json
import getpass
import requests
import urllib3
from urllib.parse import urlparse
import pandas as pd
import numpy as np
urllib3.disable_warnings()
from requests.auth import HTTPBasicAuth
from statsmodels.formula.api import ols
from datetime import datetime 
import pytz 
tz = pytz.timezone('Pacific/Auckland')

In [2]:
with open('work/appsettings.json') as c:
    config = json.load(c)
header = {
        'content-type': 'application/json',
        'X-Requested-With':'XmlHttpRequest'
    }
security_auth = HTTPBasicAuth(config['Username'], config['Password'])
piwebapi_url = config['Resource']
PIDATA_SERVER = config['DataArchiveName']
AF_SERVER = config['AFServerName']
AF_DATABASE = config['AFDatabaseName']
elementlayer = 'SMART_METER'

In [23]:
from csv import writer

def write_to_csv(log):
# Open our existing CSV file in append mode
# Create a file object for this file
    with open('work/Imp_Output.csv', 'a') as f_object:
     
        # Pass this file object to csv.writer()
        # and get a writer object
        writer_object = writer(f_object) 
        # Pass the list as an argument into
        # the writerow()
        writer_object.writerow([log])
     
        # Close the file object
        f_object.close()

In [4]:
def getelementbypath(path):
    request_url = piwebapi_url + "/elements?path=" +path
    response = requests.get(request_url, auth=security_auth, verify=False)
    if response.status_code == 200:
        json_data = response.json()
        return json_data
    else:
        return ('Error:'+ str(response.status_code) +"-" + response.reason +"-" + response.text)

In [5]:
def getattributesbypath(path):
    request_url = piwebapi_url + "/attributes?path=" +path
    response = requests.get(request_url, auth=security_auth, verify=False)
    if response.status_code == 200:
        json_data = response.json() 
        return json_data
    else:
        return ('Error:'+ str(response.status_code) +"-" + response.reason +"-" + response.text)

In [6]:
def getchildelements(parent_WebId, startindex):
    request_url = piwebapi_url + "/elements/" + parent_WebId + "/elements?selectedFields=Items.Name;Items.WebId&startIndex="+ str(startindex)
    response = requests.get(request_url, auth=security_auth, verify=False)
    if response.status_code == 200:
        json_data = response.json()  
        df = pd.json_normalize(json_data["Items"])
        return df
    else:
        return ('Error:'+ str(response.status_code) +"-" + response.reason +"-" + response.text)

In [7]:
def getpipointWebId(element_name, pipoint):
    path = '\\\\' + AF_SERVER + '\\' + AF_DATABASE + '\\' + elementlayer + '\\' + element_name + '|' + pipoint
    json= getattributesbypath(path)
    if 'Error' in json:
        Error_message = json
        return Error_message
    else:
        pipoint_webId = json['WebId']
        return pipoint_webId

In [8]:
## get data for the required Pi points
def getPipointData(pipoint_webId): 
    request_url = piwebapi_url + "/streams/" + pipoint_webId + "/recorded?startTime=t-1d&endTime=t&selectedFields=Items.Timestamp;Items.Value"
    response = requests.get(request_url, auth=security_auth, verify=False)
    if response.status_code ==200:
        json_data = response.json()
        df = pd.json_normalize(json_data['Items'])
        return df
    else:
        return ('Error:'+ str(response.status_code) +"-" + response.reason +"-" + response.text)

In [9]:
def calcurrent(p, q, v):
    return (np.sqrt((abs(p*1000)**2) + (abs(q*1000)**2)))/v

In [10]:
def calculate_current(df, metertype):
    if metertype == '1PH1E':
        ## Calculate current from P and Q values
        df['_E1.Current'] = calcurrent(df['_E1.ActivePower'] , df['_E1.ReactivePower'], df['_E1.Voltage'])
    elif metertype == '1PH2E':
        df['_E1.Current'] = calcurrent(df['_E1.ActivePower'] , df['_E1.ReactivePower'], df['_E1.Voltage'])
        df['_E2.Current'] = calcurrent(df['_E2.ActivePower'] , df['_E2.ReactivePower'], df['_E1.Voltage'])
    elif metertype == '3PH':
        df['_E1.Current'] = calcurrent(df['_E1.ActivePower'] , df['_E1.ReactivePower'], df['_E1.Voltage'])
        df['_E2.Current'] = calcurrent(df['_E2.ActivePower'] , df['_E2.ReactivePower'], df['_E2.Voltage'])
        df['_E3.Current'] = calcurrent(df['_E3.ActivePower'] , df['_E3.ReactivePower'], df['_E3.Voltage'])
    return df

In [11]:
def calculate_deltas(df, metertype):
        if metertype == '1PH2E':
            df['_E1.Current'] = df['_E1.Current']+ df['_E2.Current']
            df['dIa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Current'].diff()
            df['dVa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Voltage'].diff()
        elif metertype == '3PH':
            df['dIa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Current'].diff()
            df['dVa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Voltage'].diff()
            df['dIb'] = df[(df['period']<= 5) & ( df['_E2.ActivePower']>=0)& (df['_E2.Voltage'] >180)]['_E2.Current'].diff()
            df['dVb'] = df[(df['period']<= 5) & ( df['_E2.ActivePower']>=0)& (df['_E2.Voltage'] >180)]['_E2.Voltage'].diff()
            df['dIc'] = df[(df['period']<= 5) & ( df['_E3.ActivePower']>=0)& (df['_E3.Voltage'] >180)]['_E3.Current'].diff()
            df['dVc'] = df[(df['period']<= 5) & ( df['_E3.ActivePower']>=0)& (df['_E3.Voltage'] >180)]['_E3.Voltage'].diff()
        elif metertype == '1PH1E':
            df['dIa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Current'].diff()
            df['dVa'] = df[(df['period']<= 5) & ( df['_E1.ActivePower']>=0)& (df['_E1.Voltage'] >180)]['_E1.Voltage'].diff()       
        df.replace([np.inf, -np.inf], 0, inplace=True)
        return df

In [12]:
def calculate_regression(df, col1, col2):
    if len(df[(abs(df[col2]) >= 1)])< 5:
        message = 'Sample count less than 5'
        return message
    else:
        ols_formula = "{} ~ {} -1".format(col1.replace("'",""), col2.replace("'",""))
        lr = ols(formula = ols_formula, data = df[(abs(df[col2]) >= 1)]).fit()
        Slope = lr.params.iloc[0]
        return Slope

In [13]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [14]:
def postPIPointValue(PIPointWebID, body):
    header = {
        'content-type': 'application/json',
        'X-Requested-With':'XmlHttpRequest'
    }
    PostPIPointurl = piwebapi_url + "/streams/" + PIPointWebID + "/value" 
    post_response = requests.post(PostPIPointurl, auth=security_auth, verify=False, json=body, headers=header)
    return post_response

In [24]:
## get the count of smart meters from the smart_meter layer attribute
path="\\\\" + AF_SERVER + "\\" + AF_DATABASE + "\\"+ elementlayer+ "|SmartMeterCount"
try:
    count_attr = getattributesbypath(path)
    if 'Error' in count_attr:
        log = "SmartMeterCount Attribute not found:" + count_attr
        write_to_csv(log)
    else:
        url = urlparse(piwebapi_url + '/attributes/' + count_attr['WebId'] + '/value')
        response = requests.get(url.geturl(),auth=security_auth, verify=False)
        if response.status_code == 200:
            count_json = response.json()
            count_value = count_json['Value']
        else: 
            log = "SmartMeterCount Attribute Value:"+ 'Error:'+ str(response.status_code) +"-" + response.reason +"-" + response.text
            write_to_csv(log)
except:
    log = "Connection Error- Process Stopped"
    write_to_csv(log)

In [25]:
## get the parent WebID (for smart meter layer) for extracting list of child elements(meters)
path="\\\\" + AF_SERVER + "\\" + AF_DATABASE + "\\"+ elementlayer
try:
    d = getelementbypath(path)
    if 'error' in d:
       log = " WebId for smartmeter layer not Found:" + d
       write_to_csv(log) 
    else:
        parent_WebId = d['WebId']
        ## extract all elements/smart meters names and webIDs into a dataframe
        meter_df = pd.DataFrame()
        for i in range(0, count_value,1000):
            elem_df = getchildelements(parent_WebId, i)
            if 'Error' in elem_df:
                log = "Meter elements not extracted:" + elem_df
                write_to_csv(log)
            else:
                meter_df = pd.concat( [meter_df, elem_df], axis= 0, ignore_index=True) 
except:
    log = "Connection Error- Process Stopped"
    write_to_csv(log)

In [17]:
def calculateimpedance(element_name):
    ## get metertype or config
    path = "\\\\" + AF_SERVER + "\\" + AF_DATABASE + "\\"+ elementlayer + "\\" + element_name + "|MeterConfiguration"
    metertype_attr = getattributesbypath(path)
    if 'Error' in metertype_attr:
        log  = "Meter type attribute not found:" + element_name
        write_to_csv(log)
    else:
        url = urlparse(piwebapi_url + '/streams/' + metertype_attr['WebId'] + '/value')
        response = requests.get(url.geturl(),auth=security_auth, verify=False)
        if response.status_code == 200:
            meter_json = response.json()
            metertype = meter_json['Value']
        else: 
            log = 'Meter type Value Error for:'+ element_name  + ':' + str(response.status_code) +"-" + response.reason +"-" + response.text
            write_to_csv(log)
     ## get data for pi tags based on the meter type
    if metertype =='1PH1E':
        point_list = ['_E1.ActivePower', '_E1.Voltage',  '_E1.ReactivePower']
        df_pipointdata= pd.DataFrame()
        for pipoint in point_list:
            pipoint_webId = getpipointWebId(element_name, pipoint)
            if 'Error' in pipoint_webId:
                log = "Missing PI tag for  :" + element_name+ pipoint
                write_to_csv(log)
            else:
                df_pidata = getPipointData(pipoint_webId)
                df_pidata['pipoint'] = pipoint
                if not df_pidata.empty:
                   df_pipointdata= pd.concat([df_pipointdata, df_pidata], axis= 0, ignore_index=True)
        if df_pipointdata.empty:
            log = 'No recent data available for: '+ element_name
            write_to_csv(log)
        else:
            df_pipointdata['Timestamp'] = pd.to_datetime(df_pipointdata['Timestamp'], utc=True).dt.tz_convert('Pacific/Auckland').dt.strftime('%Y-%m-%d %H:%M:00')
            df_pipointdata['meter'] =   element_name
            df_pipointdata['metertype'] = metertype
            df_pipointdata = df_pipointdata.drop_duplicates(['pipoint','Timestamp'], keep='last')
            if "Errors" in df_pipointdata.columns:
                log = "No data available for:  "+element_name
                write_to_csv(log)
            else:
                df_data = pd.pivot(df_pipointdata, values = 'Value', index = ['meter', 'Timestamp', 'metertype'], columns = ['pipoint']).reset_index()
                if np.isin(point_list, df_data.columns).all():
                    df_data = df_data.sort_values(by='Timestamp', ascending=True).reset_index(drop=True)
                    df_data['Timestamp'] = pd.to_datetime(df_data['Timestamp'])
                    df_data = calculate_current(df_data, metertype)
                    df_data['period'] = ((df_data['Timestamp'].diff())/ np.timedelta64(1,'m')).fillna(0)
                    df_d = calculate_deltas(df_data, metertype)
                    E_Imp = calculate_regression(df_d, 'dVa', 'dIa')
                    imp_pipoint = '_E1.ImpedanceMaxCalc'
                    path = "\\\\"+ PIDATA_SERVER +"\\"+ element_name + imp_pipoint
                    point_resp = getattributesbypath(path)
                    if "Error" in point_resp:
                        log = "Impedance tag not found for " + element_name + imp_pipoint
                        write_to_csv(log)
                    else:
                        PointWebID = point_resp['WebId']
                        if is_float(E_Imp)==True:
                            body = {"Timestamp": tz.localize(pd.to_datetime(df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')), is_dst=None).astimezone('utc').strftime('%Y-%m-%dT%H:%M:%SZ'), "Value": round(abs(E_Imp),2), "Good": True}
                            postresponse = postPIPointValue(PointWebID, body)
                            if postresponse.status_code == 202:
                               log =   "Impedance Calculated for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                               write_to_csv(log)
                            else:
                                log = "Error in posting Impedance Value for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                                write_to_csv(log)
                        else:
                            log = "Impedance not calculated for " +element_name +"_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00') +" :" + E_Imp
                            write_to_csv(log)
                else:
                    log = "Missing PI tags for  :" + element_name
                    write_to_csv(log)
    elif metertype =='1PH2E':
        point_list = ['_E1.ActivePower','_E2.ActivePower', '_E1.Voltage',  '_E1.ReactivePower', '_E2.ReactivePower']
        df_pipointdata= pd.DataFrame()
        for pipoint in point_list:
            pipoint_webId = getpipointWebId(element_name, pipoint)
            if 'Error' in pipoint_webId:
                log = "Missing PI tag for  :" + element_name+ pipoint
                write_to_csv(log)
            else:
                df_pidata = getPipointData(pipoint_webId)
                df_pidata['pipoint'] = pipoint
                if not df_pidata.empty:
                   df_pipointdata= pd.concat([df_pipointdata, df_pidata], axis= 0, ignore_index=True)
        if df_pipointdata.empty:
            log= 'No recent data available for: '+ element_name
            write_to_csv(log)
        else:
            df_pipointdata['Timestamp'] = pd.to_datetime(df_pipointdata['Timestamp'], utc=True).dt.tz_convert('Pacific/Auckland').dt.strftime('%Y-%m-%d %H:%M:00')
            df_pipointdata['meter'] =   element_name
            df_pipointdata['metertype'] = metertype
            df_pipointdata = df_pipointdata.drop_duplicates(['pipoint','Timestamp'], keep='last')
            if "Errors" in df_pipointdata.columns:
                log = "No data available for:  "+element_name
                write_to_csv(log)
            else:
                df_data = pd.pivot(df_pipointdata, values = 'Value', index = ['meter', 'Timestamp', 'metertype'], columns = ['pipoint']).reset_index()
                if np.isin(point_list, df_data.columns).all():
                    df_data = df_data.sort_values(by='Timestamp', ascending=True).reset_index(drop=True)
                    df_data['Timestamp'] = pd.to_datetime(df_data['Timestamp'])
                    df_data = calculate_current(df_data, metertype)
                    df_data['period'] = ((df_data['Timestamp'].diff())/ np.timedelta64(1,'m')).fillna(0)
                    df_d = calculate_deltas(df_data, metertype)
                    E_Imp = calculate_regression(df_d, 'dVa', 'dIa')
                    imp_pipoint = '_E1.ImpedanceMaxCalc'
                    path = "\\\\"+ PIDATA_SERVER +"\\"+ element_name + imp_pipoint
                    point_resp = getattributesbypath(path)
                    if "Error" in point_resp:
                        log = "Impedance tag not found for " + element_name + imp_pipoint
                        write_to_csv(log)
                    else:
                        PointWebID = point_resp['WebId']
                        if is_float(E_Imp)==True:
                            body = {"Timestamp": tz.localize(pd.to_datetime(df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')), is_dst=None).astimezone('utc').strftime('%Y-%m-%dT%H:%M:%SZ'), "Value": round(abs(E_Imp),2), "Good": True}
                            postresponse = postPIPointValue(PointWebID, body)
                            if postresponse.status_code == 202:
                               log = "Impedance Calculated for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                               write_to_csv(log)
                            else:
                                log ="Error in posting Impedance Value for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                                write_to_csv(log)
                        else:
                            log = "Impedance not calculated for " +element_name +"_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00') +" :" + E_Imp
                            write_to_csv(log)
                else:
                    log = "Missing PI tags for  :" + element_name
                    write_to_csv(log)
    elif metertype =='3PH':    
        point_list = ['_E1.ActivePower','_E2.ActivePower','_E3.ActivePower', '_E1.Voltage', '_E2.Voltage','_E3.Voltage', '_E1.ReactivePower', '_E2.ReactivePower', '_E3.ReactivePower']
        df_pipointdata= pd.DataFrame()
        for pipoint in point_list:
            pipoint_webId = getpipointWebId(element_name, pipoint)
            if 'Error' in pipoint_webId:
                log = "Missing PI tag for  :" + element_name+ pipoint
                write_to_csv(log)
            else:
                df_pidata = getPipointData(pipoint_webId)
                df_pidata['pipoint'] = pipoint
                if not df_pidata.empty:
                   df_pipointdata= pd.concat([df_pipointdata, df_pidata], axis= 0, ignore_index=True)
        if df_pipointdata.empty:
            log = 'No recent data available for: '+ element_name
            write_to_csv(log)
        else:
            df_pipointdata['Timestamp'] = pd.to_datetime(df_pipointdata['Timestamp'], utc=True).dt.tz_convert('Pacific/Auckland').dt.strftime('%Y-%m-%d %H:%M:00')
            df_pipointdata['meter'] =   element_name
            df_pipointdata['metertype'] = metertype
            df_pipointdata = df_pipointdata.drop_duplicates(['pipoint','Timestamp'], keep='last')
            if "Errors" in df_pipointdata.columns:
                log = "No data available for:  "+element_name
                write_to_csv(log)
            else:
                df_data = pd.pivot(df_pipointdata, values = 'Value', index = ['meter', 'Timestamp', 'metertype'], columns = ['pipoint']).reset_index()
                if np.isin(point_list, df_data.columns).all():
                    df_data = df_data.sort_values(by='Timestamp', ascending=True).reset_index(drop=True)
                    df_data['Timestamp'] = pd.to_datetime(df_data['Timestamp'])
                    df_data = calculate_current(df_data, metertype)
                    df_data['period'] = ((df_data['Timestamp'].diff())/ np.timedelta64(1,'m')).fillna(0)
                    df_d = calculate_deltas(df_data, metertype)
                    E_Imp = calculate_regression(df_d, 'dVa', 'dIa')
                    imp_pipoint = '_E1.ImpedanceMaxCalc'
                    path = "\\\\"+ PIDATA_SERVER +"\\"+ element_name + imp_pipoint
                    point_resp = getattributesbypath(path)
                    if "Error" in point_resp:
                        log = "Impedance tag not found for " + element_name + imp_pipoint
                        write_to_csv(log)
                    else:
                        PointWebID = point_resp['WebId']
                        if is_float(E_Imp)==True:
                            body = {"Timestamp": tz.localize(pd.to_datetime(df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')), is_dst=None).astimezone('utc').strftime('%Y-%m-%dT%H:%M:%SZ'), "Value": round(abs(E_Imp),2), "Good": True}
                            postresponse = postPIPointValue(PointWebID, body)
                            if postresponse.status_code == 202:
                               log = "Impedance Calculated for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                               write_to_csv(log)
                            else:
                                log = "Error in posting Impedance Value for " + element_name + "_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                                write_to_csv(log)
                        else:
                            log = "Impedance not calculated for " +element_name +"_E1 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00') +" :" + E_Imp
                            write_to_csv(log)
                    E_Imp = calculate_regression(df_d, 'dVb', 'dIb')
                    imp_pipoint = '_E2.ImpedanceMaxCalc'
                    path = "\\\\"+ PIDATA_SERVER +"\\"+ element_name + imp_pipoint
                    point_resp = getattributesbypath(path)
                    if "Error" in point_resp:
                        log = "Impedance tag not found for " + element_name + imp_pipoint
                        write_to_csv(log)
                    else:
                        PointWebID = point_resp['WebId']
                        if is_float(E_Imp)==True:
                            body = {"Timestamp": tz.localize(pd.to_datetime(df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')), is_dst=None).astimezone('utc').strftime('%Y-%m-%dT%H:%M:%SZ'), "Value": round(abs(E_Imp),2), "Good": True}
                            postresponse = postPIPointValue(PointWebID, body)
                            if postresponse.status_code == 202:
                               log = "Impedance Calculated for " + element_name + "_E2 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                               write_to_csv(log)
                            else:
                                log = "Error in posting Impedance Value for " + element_name + "_E2 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                                write_to_csv(log)
                        else:
                            log = "Impedance not calculated for " +element_name +"_E2 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00') +" :" + E_Imp
                            write_to_csv(log)
                    E_Imp = calculate_regression(df_d, 'dVc', 'dIc')
                    imp_pipoint = '_E3.ImpedanceMaxCalc'
                    path = "\\\\"+ PIDATA_SERVER +"\\"+ element_name + imp_pipoint
                    point_resp = getattributesbypath(path)
                    if "Error" in point_resp:
                        log ="Impedance tag not found for " + element_name + imp_pipoint
                        write_to_csv(log)
                    else:
                        PointWebID = point_resp['WebId']
                        if is_float(E_Imp)==True:
                            body = {"Timestamp": tz.localize(pd.to_datetime(df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')), is_dst=None).astimezone('utc').strftime('%Y-%m-%dT%H:%M:%SZ'), "Value":round(abs(E_Imp),2), "Good": True}
                            postresponse = postPIPointValue(PointWebID, body)
                            if postresponse.status_code == 202:
                               log  = "Impedance Calculated for " + element_name + "_E3 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                               write_to_csv(log)
                            else:
                                log =  "Error in posting Impedance Value for " + element_name + "_E3 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00')
                                write_to_csv(log)
                        else:
                            log = "Impedance not calculated for " +element_name +"_E3 date: " +df_data['Timestamp'].max().strftime('%Y-%m-%d 23:55:00') +" :" + E_Imp
                            write_to_csv(log)
                else:
                    log = "Missing PI tags for  :" + element_name
                    write_to_csv(log)
    else:
        log ="Invalid metertype for: "+ element_name
        write_to_csv(log)
        

In [18]:
#meter_df[meter_df['Name']== 'meter.']
#test_list= meter_df.iloc[57665:57680]['Name'].to_list()
list = meter_df['Name'].to_list()
len(list)

71000

In [19]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from concurrent.futures import wait
from time import time
import asyncio
import multiprocessing

In [20]:
'''
start = time()
log = 'Time started: ' + str(start)
write_to_csv([log])

with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(calculateimpedance, list)

executor.shutdown(wait=True, cancel_futures=False)

log = 'Time taken: ' + str(time() - start)
write_to_csv([log])
'''

"\nstart = time()\nlog = 'Time started: ' + str(start)\nwrite_to_csv([log])\n\nwith ThreadPoolExecutor(max_workers=10) as executor:\n    executor.map(calculateimpedance, list)\n\nexecutor.shutdown(wait=True, cancel_futures=False)\n\nlog = 'Time taken: ' + str(time() - start)\nwrite_to_csv([log])\n"

In [21]:
def parallel_processing(list, pool_size):
    # Create a multiprocessing pool
    pool = multiprocessing.Pool(pool_size)

    # Use the map function to distribute the work to the pool of processes
    results = pool.map(calculateimpedance, list)

    # Close the pool and wait for all tasks to complete
    pool.close()
    #pool.join()
    return results

In [ ]:
if __name__ == "__main__":
    # Specify the number of processes to use (adjust according to your CPU cores)
    pool_size = multiprocessing.cpu_count()
    # Perform the parallel processing
    results = parallel_processing(list, 10)